in the standard approach, Trainer API is used. 
Multi label classification

All parameters of the pre-trained models are trained this includs:  

    Embedding layer (token, position and segment embedding)
    attention weights in transformer layers
    feedforward neural netwrok wieghts in each layer
    final classification/regression head 

In [1]:
from transformers import AutoModel
model = AutoModel.from_pretrained("bio_clinical_bert")

In [2]:
#print(model)

In [3]:
# from transformers import BertConfig
# config = BertConfig.from_pretrained("bio_clinical_bert")
# print(config)

In [2]:
from datasets import load_dataset
dataset = load_dataset("ricardosantoss/mimic")

In [3]:
print(dataset['train'][0])

{'TEXT': "Admission Date: [**2117-9-11**] Discharge Date: [**2117-9-17**] Date Birth: [**2082-3-21**] Sex: F Service: MEDICINE Allergies: Levaquin Attending:[**First Name3 (LF) 2195**] Chief Complaint: nausea, vomiting Major Surgical Invasive Procedure: none History Present Illness: 35F w/ poorly controlled Type 1 diabetes mellitus w/ neuropathy, nephropathy, HTN, gastroparesis, CKD retinopathy, recently hospitalized orthostatic hypotension [**2-3**] autonomic neuropathy [**Date range (1) 25088**]; DKA hospitalizations [**6-12**] [**7-12**], returning w/ 5d history worsening nausea, vomiting coffee-ground emesis, chills, dyspnea exertion. Last week fall hit right face. also 1 day diarrhea, resolved early last week. Found DKA AG 30 bicarb 11. . ED inital vitals 09:00 0 98.2 113 181/99 22 100% RA. K 4.7, HCO3 11, Anion Gap 30, Cr. 2.7 (baseline 1.6-2.0) 3rd L NS. Insulin srip 5 units/hr. home 22 levemir 12 difficult control sugars. BPs high. Given 30 mtroprolol tartrate ED. started insul

In [4]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-cased")

/home/shiva/.local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
# from sklearn.preprocessing import LabelEncoder
# from itertools import chain

# # Extract all ICD9 codes and encode them
# all_codes = list(chain.from_iterable(dataset["train"]["ICD9_CODE"]))
# label_encoder = LabelEncoder()
# label_encoder.fit(all_codes)

LabelEncoder()

In [7]:
print(dataset.column_names)

{'train': ['TEXT', 'ICD9_CODE'], 'test': ['TEXT', 'ICD9_CODE'], 'validation': ['TEXT', 'ICD9_CODE']}


In [6]:
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer

In [10]:
mlb = MultiLabelBinarizer()
all_icd_codes = np.concatenate(dataset["train"]["ICD9_CODE"] + 
                  dataset["test"]["ICD9_CODE"] + 
                  dataset["validation"]["ICD9_CODE"])
mlb.fit([all_icd_codes])

MultiLabelBinarizer()

In [11]:
def binarize_labels(example):
    example["labels"] = mlb.transform([example["ICD9_CODE"]])[0].astype(float)
    return example

In [12]:
print("unique ICD-9 classes", mlb.classes_)
print("binary lables_shape", all_icd_codes.shape)

unique ICD-9 classes ['0389' '2449' '25000' '2720' '2724' '2760' '2761' '2762' '2767' '2851'
 '2859' '2875' '3051' '311' '32723' '4019' '40390' '40391' '41071' '412'
 '41401' '4168' '4240' '4241' '42731' '42789' '4280' '486' '49390' '496'
 '5070' '5119' '5180' '51881' '53081' '5845' '5849' '5859' '5990' '7742'
 '78552' '99592' '9971' 'V053' 'V1582' 'V290' 'V4581' 'V4582' 'V5861'
 'V5867']
binary lables_shape (228387,)


In [38]:
dataset_train = dataset["train"].map(binarize_labels)

Map:   0%|          | 0/39354 [00:00<?, ? examples/s]

In [39]:
dataset_train = dataset["test"].map(binarize_labels)

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [41]:
dataset_train = dataset["validation"].map(binarize_labels)

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [42]:
def tokenize(examples):
    return tokenizer(
        examples["TEXT"],
        padding="max_length",
        truncation=True,
        max_length=512,
        return_tensors="pt"
        
    )
    # Convert list of ICD9_CODE strings to integer indices
    
    

In [43]:
tokenized_dataset_train = dataset["train"].map(tokenize, batched=True)

Map:   0%|          | 0/39354 [00:00<?, ? examples/s]

In [44]:
tokenized_dataset_test = dataset["test"].map(tokenize, batched=True)

In [46]:
tokenized_dataset_val = dataset["validation"].map(tokenize, batched=True)

In [ ]:
# print(dataset.column_names)

{'train': ['TEXT', 'ICD9_CODE'], 'test': ['TEXT', 'ICD9_CODE'], 'validation': ['TEXT', 'ICD9_CODE']}


In [15]:
# test_ds = dataset["test"]
# print(test_ds.column_names)

In [ ]:
# small_test = test_ds.select(range(10))

In [ ]:
# train_ds = dataset["train"]

In [ ]:
# small_train = train_ds.select(range(10))

In [ ]:
# print(train_ds.column_names)

['TEXT', 'ICD9_CODE']


In [ ]:
# tokenized_train = small_train.map(tokenize)

In [37]:
# print(tokenized_train.column_names )

In [36]:
# print(tokenized_train)

In [ ]:
# tokenized_test = small_test.map(tokenize)

In [ ]:
# tokenized_test = tokenized_test.remove_columns(["TEXT", "ICD9_CODE"])

In [ ]:
# tokenized_train = tokenized_train.remove_columns(["TEXT", "ICD9_CODE"])

In [ ]:
<!-- from itertools import chain

all_codes = list(chain.from_iterable(dataset["train"]["ICD9_CODE"]))
num_classes = len(set(all_codes))
print("Number of classes:", num_classes) -->

Number of classes: 50


In [20]:
from transformers import AutoModelForSequenceClassification
# cls_model = AutoModelForSequenceClassification.from_pretrained("bio_clinical_bert", num_labels=num_classes)

In [29]:
multi_cls_model = AutoModelForSequenceClassification.from_pretrained(
    "emilyalsentzer/Bio_ClinicalBERT",
    num_labels=len(mlb.classes_),  # Number of unique ICD-9 codes
    problem_type="multi_label_classification"  # Critical for correct loss
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at emilyalsentzer/Bio_ClinicalBERT and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [47]:
print(multi_cls_model.config.problem_type)

multi_label_classification


In [35]:
# print(tokenized_train.column_names)

In [ ]:
# import torch

# # Display the output structure of cls_model for a batch of tokenized inputs
# sample = tokenized_train[0]
# inputs = {k: [sample[k]] for k in ['input_ids', 'attention_mask', 'token_type_ids']}
# with torch.no_grad():
#     outputs = cls_model(**{k: torch.tensor(v) for k, v in inputs.items()})
# print(outputs)

SequenceClassifierOutput(loss=None, logits=tensor([[-0.2533,  0.0144,  0.3630, -0.0816, -0.2614,  0.4188,  0.3951,  0.4018,
         -0.1115,  0.1536, -0.2904,  0.1073, -0.5302,  0.5028,  0.2469, -0.3734,
         -0.1077,  0.2747,  0.1318,  0.0073,  0.4400, -0.1190, -0.2135, -0.0772,
          0.5145,  0.4516, -0.3133,  0.1246,  0.6887,  0.4123, -0.0482, -0.2660,
          0.1493,  0.0693, -0.1280,  0.0171, -0.1513,  0.2029, -0.5039, -0.4185,
          0.0675,  0.4843, -0.2173,  0.3385, -0.2675, -0.1958,  0.2090,  0.2946,
          0.3498, -0.4317]]), hidden_states=None, attentions=None)


In [16]:
from sklearn.metrics import f1_score

In [17]:
#!pip install evaluate
import evaluate
import numpy as np

metric = evaluate.load("accuracy")
def compute_metric(eval_pred):
    logits, labels = eval_pred
    predictions = (logits > 0).astype(int)
    return {
        "micro_f1": f1_score(labels, predictions, average="micro"),
        "macro_f1": f1_score(labels, predictions, average="macro"),
    }


/home/shiva/.local/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/shiva/.local/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [48]:
from transformers import TrainingArguments
training_args = TrainingArguments(

    output_dir="outputs_clical_bert", 
    eval_strategy="epoch",
    save_strategy="epoch",
    metric_for_best_model="micro_f1",
    load_best_model_at_end=True
)

In [49]:
from transformers import Trainer

trainer = Trainer(
    model = multi_cls_model, 
    args=training_args, 
    train_dataset=tokenized_dataset_train, 
    eval_dataset=tokenized_dataset_test, 
    compute_metrics=compute_metric
)

In [24]:
# print(tokenized_train)
# print(tokenized_test)

In [ ]:

# train_dataset = tokenized_train.remove_columns("token_type_ids")
# test_dataset = tokenized_test.remove_columns("token_type_ids")


In [ ]:
# train_dataset.column_names

['input_ids', 'attention_mask', 'labels']

In [ ]:
# train_dataset["labels"]

[[36, 49, 16],
 [9, 15],
 [33, 27, 6, 44],
 [27, 15],
 [20, 2, 47, 44, 10, 15, 3, 49],
 [9],
 [20, 15, 3, 44, 47],
 [30, 33],
 [6, 31, 26, 24, 48, 20, 47, 29, 19],
 [23]]

In [25]:
# train_dataset = train_dataset.map(lambda x: {"labels": int(x["labels"])})

In [50]:
trainer.train()

  0%|          | 0/14760 [00:00<?, ?it/s]

ValueError: The model did not return a loss from the inputs, only the following keys: logits. For reference, the inputs it received are input_ids,token_type_ids,attention_mask.

In [34]:
print(tokenized_dataset["train"].features)
print(tokenized_dataset["train"][0]["labels"]) 

{'TEXT': Value(dtype='string', id=None), 'ICD9_CODE': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None), 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None), 'token_type_ids': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None), 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}


KeyError: 'labels'

In [ ]:
def tokenize(examples):
    tokenized = tokenizer(
        examples["TEXT"],
        padding="max_length",
        truncation=True,
        return_attention_mask=True,
        return_token_type_ids=True
    )
    # Multi-label: create a binary vector for each example
    labels = np.zeros(num_classes, dtype=int)
    for code in examples["ICD9_CODE"]:
        idx = label_encoder.transform([code])[0]
        labels[idx] = 1
    tokenized["labels"] = labels
    return tokenized

In [36]:
print(tokenized_train.column_names)

['input_ids', 'token_type_ids', 'attention_mask', 'labels']
